## Removing useless nonterminal symbols and identifying nullable (vanishing) symbols for context free grammar

We are given a context free grammar $G(\Sigma, N, S \in N, P)$

Our task is to remove all useless non-terminal symbols, and to identify nullable (vanishing) non-terminal symbols. 

To start with, we will use the following context free grammar structure:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
```

Let's implement a method that returns all productive (generating) symbols.

A nonterminal symbol $X$ is called productive, if $\forall \alpha, \beta \in (\Sigma \cup N)^* \space \exists u \in \Sigma : (\alpha X \beta) \twoheadrightarrow u$

We will expand the set of productive symbols as it grows.

$E_0 = \emptyset$

$E_1 = \{X \in N | \space \exists u \in \Sigma^* , (X \to u) \in P\}$
$E_2 = \{X \in N | \space \exists k \in \mathbb{N} \space \exists \alpha_1, \alpha_1, ..., \alpha_k \in (\Sigma \cup E_1), (X \to \alpha_1\alpha_1...\alpha_k) \in P\}$

...

$E_{n+1} = \{X \in N | \space \exists k \in \mathbb{N} \space \exists \alpha_1, \alpha_1, ..., \alpha_k \in (\Sigma \cup E_{n}), (X ]to \alpha_1\alpha_1...\alpha_k) \in P\}$

$E_n = E_{n+1} \supset N$ - the desired set of productive symbols.
$N - E_n$ - the desired set of unproductive symbols.

In [25]:
def productive_symbols(grammar) -> set:
  productive = set()
  prev_count = None

  def absent_unproductive_symbols(rule: str) -> bool:
    return all(map(lambda s: s in productive or s in grammar['toks'], rule))

  while len(productive) != prev_count:
    prev_count = len(productive)
    productive_symbols = [var for var, definition in grammar['vars'].items() if list(filter(absent_unproductive_symbols, definition)) != []]
    productive = productive.union(set(productive_symbols))

  return productive  

Let's implement a method that returns all reachable symbols.

A nonterminal symbol $X$ is called reachable, if $\exists \alpha, \beta \in (\Sigma \cup N)^* \space : S \twoheadrightarrow (\alpha X \beta)$

We will expand the set of reachable symbols as it grows.

$E_0 = \emptyset$

$E_1 = \{X \in N | \space \exists \alpha, \beta \in (\Sigma \cup N)^*: (S \to \alpha X \beta) \in P\}$
$E_2 = \{X \in N | \exists M \in E_1 \space \exists \alpha, \beta \in (\Sigma \cup N)^*: (M \to \alpha X \beta) \in P\}$

...

$E_{n+1} = \{X \in N | \exists M \in E_n \space \exists \alpha, \beta \in (\Sigma \cup N)^*: (M \to \alpha X \beta) \in P\}$

$E_n = E_{n+1} \supset N$ - the desired set of reachable symbols.
$N - E_n$ - the desired set of unreachable symbols.

In [26]:
def reachable_symbols(grammar) -> set:
  reachable = {grammar['hvar']}
  prev_count = None

  while len(reachable) != prev_count:
    prev_count = len(reachable)
    for var, definition in grammar['vars'].items():
      if var in reachable:
        for rule in definition:
          reachable = reachable.union(set(filter(lambda s: s not in grammar['toks'], rule)))

  return reachable  

Let's implement a method that remove all useless non-terminal symbols. For this we remove all rules with unproductive or unreachable symbols.

In [27]:
from copy import deepcopy

def remove_useless_symbols(grammar):
  grammar_copy = deepcopy(grammar)

  productive = productive_symbols(grammar)
  grammar_copy['vars'] = {var: [rule for rule in definition if all(map(lambda s: s in productive or s in grammar['toks'], rule))] 
                          for var, definition in grammar_copy['vars'].items()}

  reachable = reachable_symbols(grammar_copy)
  grammar_copy['vars'] = {var: [rule for rule in definition if all(map(lambda s: s in reachable or s in grammar['toks'], rule))] 
                          for var, definition in grammar_copy['vars'].items() if var in reachable}

  return grammar_copy

Let's implement a method that returns all nullable (vanishing) symbols.

A nonterminal symbol $X$ is called nullable, if $X \twoheadrightarrow e$.

We will expand the set of nullable symbols as it grows.

$V_0 = \emptyset$

$V_1 = \{X \in N | \space (X \to e) \in P\}$
$V_2 = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_1, (X \to B_1 B_2 ... B_k) \in P\}$

...

$V_{n+1} = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_n, (X \to B_1 B_2 ... B_k) \in P\}$

$V_n = V_{n+1} \supset N$ - the desired set of nullable symbols.

In [28]:
def nullable_symbols(grammar) -> set:
  nullable = set()
  prev_count = None

  def all_nullable_symbols(rule: str) -> bool:
    return all(map(lambda s: s in nullable or s == (0, ''), rule))

  while len(nullable) != prev_count:
    prev_count = len(nullable)
    nullable_symbols = [var for var, definition in grammar['vars'].items() 
                        if list(filter(all_nullable_symbols, definition)) != []]
    nullable = nullable.union(set(nullable_symbols))

  return nullable  

In [29]:
def test():
  grammar = {
      'toks': {(0, ''), (1, 'a'), (2, 'b'), (3, 'c')},
      'vars': {
          'S': [[(1, 'a'), 'M'],
                [(1, 'a'), 'S'],
                ['K']], 
          'K': [[(3, 'c')], 
                [(0, '')]], 
          'M': [['M', 'P']],
          'P': [[(3, 'c')]]
          },
      'hvar': 'S'
      }

  grammar = remove_useless_symbols(grammar)
  print(grammar)

  print('Nullable symbols are ' + str(nullable_symbols(grammar)))

test()  

{'toks': {(1, 'a'), (3, 'c'), (0, ''), (2, 'b')}, 'vars': {'S': [[(1, 'a'), 'S'], ['K']], 'K': [[(3, 'c')], [(0, '')]]}, 'hvar': 'S'}
Nullable symbols are {'S', 'K'}
